In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
%matplotlib inline

2024-12-06 14:06:39.879150: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-06 14:06:41.256683: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-06 14:06:41.873480: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-06 14:06:42.936341: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-06 14:06:43.153906: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-06 14:06:45.192345: I tensorflow/core/platform/cpu_feature_gu

In [2]:
df = pd.read_csv("transaction_dataset.csv")
df.columns
df.head()

,Index,Address,FLAG,Avg min between sent tnx,Avg min between received tnx,Time Diff between first and last (Mins),Sent tnx,Received Tnx,Number of Created Contracts,Unique Received From Addresses,...,ERC20 min val sent,ERC20 max val sent,ERC20 avg val sent,ERC20 min val sent contract,ERC20 max val sent contract,ERC20 avg val sent contract,ERC20 uniq sent token name,ERC20 uniq rec token name,ERC20 most sent token type,ERC20_most_rec_token_type
0,1,0x00009277775ac7d0d59eaad8fee3d10ac6c805e8,0,844.26,1093.71,704785.63,721,89,0,40,...,0.000000,1.683100e+07,271779.920000,0.0,0.0,0.0,39.0,57.0,Cofoundit,Numeraire
1,2,0x0002b44ddb1476db43c868bd494422ee4c136fed,0,12709.07,2958.44,1218216.73,94,8,0,5,...,2.260809,2.260809e+00,2.260809,0.0,0.0,0.0,1.0,7.0,Livepeer Token,Livepeer Token
2,3,0x0002bda54cb772d040f779e88eb453cac0daa244,0,246194.54,2434.02,516729.30,2,10,0,10,...,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,8.0,NaN,XENON
3,4,0x00038e6ba2fd5c09aedb96697c8d7b8fa6632e5e,0,10219.60,15785.09,397555.90,25,9,0,7,...,100.000000,9.029231e+03,3804.076893,0.0,0.0,0.0,1.0,11.0,Raiden,XENON
4,5,0x00062d1dd1afb6fb02540ddad9cdebfe568e0d89,0,36.61,10707.77,382472.42,4598,20,1,7,...,0.000000,4.500000e+04,13726.659220,0.0,0.0,0.0,6.0,27.0,StatusNetwork,EOS


In [3]:
np.random.seed(20)

In [4]:
df.isnull().sum()

Index                                                      0
Address                                                    0
FLAG                                                       0
Avg min between sent tnx                                   0
Avg min between received tnx                               0
Time Diff between first and last (Mins)                    0
Sent tnx                                                   0
Received Tnx                                               0
Number of Created Contracts                                0
Unique Received From Addresses                             0
Unique Sent To Addresses                                   0
min value received                                         0
max value received                                         0
avg val received                                           0
min val sent                                               0
max val sent                                               0
avg val sent            

In [5]:
df.columns

Index(['Index', 'Address', 'FLAG', 'Avg min between sent tnx',
       'Avg min between received tnx',
       'Time Diff between first and last (Mins)', 'Sent tnx', 'Received Tnx',
       'Number of Created Contracts', 'Unique Received From Addresses',
       'Unique Sent To Addresses', 'min value received', 'max value received ',
       'avg val received', 'min val sent', 'max val sent', 'avg val sent',
       'min value sent to contract', 'max val sent to contract',
       'avg value sent to contract',
       'total transactions (including tnx to create contract',
       'total Ether sent', 'total ether received',
       'total ether sent contracts', 'total ether balance',
       ' Total ERC20 tnxs', ' ERC20 total Ether received',
       ' ERC20 total ether sent', ' ERC20 total Ether sent contract',
       ' ERC20 uniq sent addr', ' ERC20 uniq rec addr',
       ' ERC20 uniq sent addr.1', ' ERC20 uniq rec contract addr',
       ' ERC20 avg time between sent tnx', ' ERC20 avg time betwe

In [6]:
dropcols = [' ERC20_most_rec_token_type',' ERC20 most sent token type','Address']
df_phase1 = df.drop(dropcols, axis = 1)   #removing cols with most NaN values

In [7]:
df_phase2 = df_phase1.dropna( axis=0)  #remove rows with NaN values
df_phase2

,Index,FLAG,Avg min between sent tnx,Avg min between received tnx,Time Diff between first and last (Mins),Sent tnx,Received Tnx,Number of Created Contracts,Unique Received From Addresses,Unique Sent To Addresses,...,ERC20 max val rec,ERC20 avg val rec,ERC20 min val sent,ERC20 max val sent,ERC20 avg val sent,ERC20 min val sent contract,ERC20 max val sent contract,ERC20 avg val sent contract,ERC20 uniq sent token name,ERC20 uniq rec token name
0,1,0,844.26,1093.71,704785.63,721,89,0,40,118,...,1.500000e+07,265586.147600,0.000000,1.683100e+07,271779.920000,0.0,0.0,0.0,39.0,57.0
1,2,0,12709.07,2958.44,1218216.73,94,8,0,5,14,...,3.650000e+02,57.632615,2.260809,2.260809e+00,2.260809,0.0,0.0,0.0,1.0,7.0
2,3,0,246194.54,2434.02,516729.30,2,10,0,10,2,...,4.428198e+02,65.189009,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,8.0
3,4,0,10219.60,15785.09,397555.90,25,9,0,7,13,...,1.141223e+04,1555.550174,100.000000,9.029231e+03,3804.076893,0.0,0.0,0.0,1.0,11.0
4,5,0,36.61,10707.77,382472.42,4598,20,1,7,19,...,9.000000e+04,4934.232147,0.000000,4.500000e+04,13726.659220,0.0,0.0,0.0,6.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9832,2171,1,0.00,0.00,0.00,0,0,0,0,0,...,1.337000e+01,6.685000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,2.0
9836,2175,1,12635.10,631.39,58748.48,4,13,0,11,4,...,1.337000e+00,0.668500,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,2.0
9837,2176,1,0.00,0.00,0.00,0,0,0,0,0,...,1.337000e+01,13.370000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,1.0
9838,2177,1,2499.44,2189.29,261601.88,67,43,0,31,44,...,1.500000e+03,300.693901,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,5.0


In [8]:
df_phase2.isnull().sum()

Index                                                   0
FLAG                                                    0
Avg min between sent tnx                                0
Avg min between received tnx                            0
Time Diff between first and last (Mins)                 0
Sent tnx                                                0
Received Tnx                                            0
Number of Created Contracts                             0
Unique Received From Addresses                          0
Unique Sent To Addresses                                0
min value received                                      0
max value received                                      0
avg val received                                        0
min val sent                                            0
max val sent                                            0
avg val sent                                            0
min value sent to contract                              0
max val sent t

In [9]:
df_phase3=df_phase2
X = df_phase2.drop('FLAG', axis=1)
Y = df_phase3['FLAG']

In [10]:
Y = pd.DataFrame(Y, columns=['FLAG'])

In [11]:
Y

,FLAG
0,0
1,0
2,0
3,0
4,0
...,...
9832,1
9836,1
9837,1
9838,1


In [12]:
Y['FLAG'].value_counts()

FLAG
0    7662
1    1350
Name: count, dtype: int64

In [13]:
cols = X.columns

In [14]:
X = df_phase3.drop('FLAG',axis=1)
X

,Index,Avg min between sent tnx,Avg min between received tnx,Time Diff between first and last (Mins),Sent tnx,Received Tnx,Number of Created Contracts,Unique Received From Addresses,Unique Sent To Addresses,min value received,...,ERC20 max val rec,ERC20 avg val rec,ERC20 min val sent,ERC20 max val sent,ERC20 avg val sent,ERC20 min val sent contract,ERC20 max val sent contract,ERC20 avg val sent contract,ERC20 uniq sent token name,ERC20 uniq rec token name
0,1,844.26,1093.71,704785.63,721,89,0,40,118,0.000000,...,1.500000e+07,265586.147600,0.000000,1.683100e+07,271779.920000,0.0,0.0,0.0,39.0,57.0
1,2,12709.07,2958.44,1218216.73,94,8,0,5,14,0.000000,...,3.650000e+02,57.632615,2.260809,2.260809e+00,2.260809,0.0,0.0,0.0,1.0,7.0
2,3,246194.54,2434.02,516729.30,2,10,0,10,2,0.113119,...,4.428198e+02,65.189009,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,8.0
3,4,10219.60,15785.09,397555.90,25,9,0,7,13,0.000000,...,1.141223e+04,1555.550174,100.000000,9.029231e+03,3804.076893,0.0,0.0,0.0,1.0,11.0
4,5,36.61,10707.77,382472.42,4598,20,1,7,19,0.000000,...,9.000000e+04,4934.232147,0.000000,4.500000e+04,13726.659220,0.0,0.0,0.0,6.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9832,2171,0.00,0.00,0.00,0,0,0,0,0,0.000000,...,1.337000e+01,6.685000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,2.0
9836,2175,12635.10,631.39,58748.48,4,13,0,11,4,0.004082,...,1.337000e+00,0.668500,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,2.0
9837,2176,0.00,0.00,0.00,0,0,0,0,0,0.000000,...,1.337000e+01,13.370000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,1.0
9838,2177,2499.44,2189.29,261601.88,67,43,0,31,44,0.001078,...,1.500000e+03,300.693901,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,5.0


In [15]:
Y

,FLAG
0,0
1,0
2,0
3,0
4,0
...,...
9832,1
9836,1
9837,1
9838,1


In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)
X_train

,Index,Avg min between sent tnx,Avg min between received tnx,Time Diff between first and last (Mins),Sent tnx,Received Tnx,Number of Created Contracts,Unique Received From Addresses,Unique Sent To Addresses,min value received,...,ERC20 max val rec,ERC20 avg val rec,ERC20 min val sent,ERC20 max val sent,ERC20 avg val sent,ERC20 min val sent contract,ERC20 max val sent contract,ERC20 avg val sent contract,ERC20 uniq sent token name,ERC20 uniq rec token name
1864,1865,0.00,1441.31,198901.02,0,138,1,3,0,0.000000,...,1.216984,1.216984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2706,2707,0.00,22235.24,200117.20,0,9,1,4,0,0.000000,...,1.043993,0.892978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
8261,600,0.00,461.00,186923.58,1,6,0,5,1,0.497000,...,13.370000,13.370000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5608,2676,162.67,0.82,326.97,2,2,0,2,2,29.374791,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8245,584,0.00,0.00,0.00,0,0,0,0,0,0.000000,...,13.370000,13.370000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,2802,0.00,16197.17,226760.43,0,14,1,6,0,0.000000,...,0.705159,0.611924,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
5191,2259,0.00,0.00,15369.12,1,1,0,1,1,2.000000,...,312.430205,50.497598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
5390,2458,0.00,0.00,1.77,1,1,0,1,1,1.990000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
860,861,165.15,0.00,330.30,2,2,0,2,2,49.770407,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
X_test

,Index,Avg min between sent tnx,Avg min between received tnx,Time Diff between first and last (Mins),Sent tnx,Received Tnx,Number of Created Contracts,Unique Received From Addresses,Unique Sent To Addresses,min value received,...,ERC20 max val rec,ERC20 avg val rec,ERC20 min val sent,ERC20 max val sent,ERC20 avg val sent,ERC20 min val sent contract,ERC20 max val sent contract,ERC20 avg val sent contract,ERC20 uniq sent token name,ERC20 uniq rec token name
5235,2303,43404.99,13493.22,184187.87,3,4,0,4,1,0.006679,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2484,2485,0.00,0.00,5.92,1,1,0,1,1,22.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2508,2509,0.00,2921.88,105187.75,0,36,1,4,0,0.000000,...,2.173992,2.173992,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0
7303,4371,544.56,26.82,6070.63,11,3,0,2,3,0.099000,...,3782.703000,2391.351500,500.0,4282.703000,2391.351500,0.0,0.0,0.0,1.0,1.0
1188,1189,40824.57,0.00,163298.27,4,0,0,0,3,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5333,2401,0.00,2061.68,226784.83,0,110,1,2,0,0.000000,...,0.803455,0.528856,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,2.0
1437,1438,9811.44,21470.73,434254.62,18,12,0,5,9,0.000830,...,549.990000,92.179768,0.0,549.990000,113.669652,0.0,0.0,0.0,6.0,13.0
2199,2200,0.00,10442.99,20885.98,0,2,0,1,0,0.000000,...,6389.006973,4444.503487,2500.0,6389.006973,4444.503487,0.0,0.0,0.0,1.0,1.0
9274,1613,0.00,0.00,3777.47,1,1,0,1,1,0.062111,...,13.370000,13.370000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0


In [18]:
Y_train

,FLAG
1864,0
2706,0
8261,1
5608,0
8245,1
...,...
5734,0
5191,0
5390,0
860,0


In [19]:
Y_test

,FLAG
5235,0
2484,0
2508,0
7303,0
1188,0
...,...
5333,0
1437,0
2199,0
9274,1


In [20]:
from sklearn.preprocessing import StandardScaler 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [21]:
X_train

array([[-0.00992642, -0.24414245, -0.29661512, ...,  0.        ,
        -0.20352484, -0.22414151],
       [ 0.66771512, -0.24414245,  0.58827704, ...,  0.        ,
        -0.20352484, -0.16780134],
       [-1.02799835, -0.24414245, -0.33833252, ...,  0.        ,
        -0.20352484, -0.22414151],
       ...,
       [ 0.46731994, -0.24414245, -0.35795052, ...,  0.        ,
        -0.20352484, -0.28048169],
       [-0.81794556, -0.23652527, -0.35795052, ...,  0.        ,
        -0.20352484, -0.28048169],
       [ 1.98034382, -0.24404836, -0.35794584, ...,  0.        ,
        -0.20352484, -0.28048169]])

In [22]:
X_test

array([[ 0.32711948,  1.576166  ,  0.22720725, ...,  0.        ,
        -0.2284832 , -0.36988925],
       [ 0.47379411, -0.24245129, -0.32565769, ...,  0.        ,
        -0.2284832 , -0.36988925],
       [ 0.49313582, -0.24245129, -0.20593793, ...,  0.        ,
        -0.2284832 , -0.28224169],
       ...,
       [ 0.24411131, -0.24245129,  0.10222854, ...,  0.        ,
        -0.02898915, -0.28224169],
       [-0.22895467, -0.24245129, -0.32565769, ...,  0.        ,
        -0.2284832 , -0.28224169],
       [-1.43861741, -0.24134516, -0.3002291 , ...,  0.        ,
        -0.2284832 , -0.28224169]])

In [23]:
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [24]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score

# Initialize models
svm_linear_model = SVC(kernel="linear", probability=True)  # Enable probability for ROC AUC
svm_rbf_model = SVC(kernel="rbf", probability=True)
svm_sigmoid_model = SVC(kernel="sigmoid", probability=True)
mlp_model = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=200, random_state=42)
xgc = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', n_estimators=100)
dctgini = DecisionTreeClassifier(criterion='gini')
dctent = DecisionTreeClassifier(criterion='entropy')
rfc = RandomForestClassifier(n_estimators=200)

# Store models in a dictionary
models = {
    "SVM Linear": svm_linear_model,
    "SVM RBF": svm_rbf_model,
    "SVM Sigmoid": svm_sigmoid_model,
    "MLP": mlp_model,
    "XGBoost": xgc,
    "Decision Tree (Gini)": dctgini,
    "Decision Tree (Entropy)": dctent,
    "Random Forest": rfc
}


In [25]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score

# Assuming X_train, X_test, Y_train, Y_test are defined
# Also assuming the models have been defined earlier

# Dictionary to store results
results = {}

# Loop through each model
for model_name, model in models.items():
    # Train the model
    model.fit(X_train, Y_train)
    
    # Get predictions
    y_pred = model.predict(X_test)
    
    # If the model supports probability predictions, compute ROC AUC
    if hasattr(model, "predict_proba"):
        y_pred_prob = model.predict_proba(X_test)[:, 1]
        roc_auc = roc_auc_score(Y_test, y_pred_prob)
    else:
        roc_auc = None  # Not all models support predict_proba
    
    # Calculate metrics
    accuracy = accuracy_score(Y_test, y_pred)
    f1 = f1_score(Y_test, y_pred)
    precision = precision_score(Y_test, y_pred)
    recall = recall_score(Y_test, y_pred)
    
    # Store metrics in the results dictionary
    results[model_name] = {
        "Accuracy": accuracy,
        "F1 Score": f1,
        "Precision": precision,
        "Recall": recall,
        "ROC AUC": roc_auc
    }

# Print results
for model_name, metrics in results.items():
    print(f"{model_name}:")
    print(f"  Accuracy: {metrics['Accuracy']:.4f}")
    print(f"  F1 Score: {metrics['F1 Score']:.4f}")
    print(f"  Precision: {metrics['Precision']:.4f}")
    print(f"  Recall: {metrics['Recall']:.4f}")
    if metrics["ROC AUC"] is not None:
        print(f"  ROC AUC: {metrics['ROC AUC']:.4f}")
    else:
        print("  ROC AUC: Not available (model does not support predict_proba)")
    print()


/home/khuushi/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/khuushi/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/khuushi/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/khuushi/.local/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1105: DataConversionWarning: A column-vector y was passed when a 1d array was expe

SVM Linear:
  Accuracy: 0.8663
  F1 Score: 0.0398
  Precision: 0.5556
  Recall: 0.0207
  ROC AUC: 0.8329

SVM RBF:
  Accuracy: 0.8702
  F1 Score: 0.0714
  Precision: 0.9000
  Recall: 0.0372
  ROC AUC: 0.9462

SVM Sigmoid:
  Accuracy: 0.8125
  F1 Score: 0.0000
  Precision: 0.0000
  Recall: 0.0000
  ROC AUC: 0.5671

MLP:
  Accuracy: 0.9196
  F1 Score: 0.7627
  Precision: 0.6314
  Recall: 0.9628
  ROC AUC: 0.9759

XGBoost:
  Accuracy: 0.9506
  F1 Score: 0.8355
  Precision: 0.7559
  Recall: 0.9339
  ROC AUC: 0.9835

Decision Tree (Gini):
  Accuracy: 0.7277
  F1 Score: 0.2243
  Precision: 0.1816
  Recall: 0.2934
  ROC AUC: 0.5442

Decision Tree (Entropy):
  Accuracy: 0.9107
  F1 Score: 0.6005
  Precision: 0.7516
  Recall: 0.5000
  ROC AUC: 0.7372

Random Forest:
  Accuracy: 0.9024
  F1 Score: 0.4395
  Precision: 0.9583
  Recall: 0.2851
  ROC AUC: 0.9463



In [26]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
import numpy as np

# Assuming X_train, X_test, Y_train, Y_test, and models are defined

# Dictionary to store results
results = {model_name: {"Accuracy": [], "F1 Score": [], "Precision": [], "Recall": [], "ROC AUC": []} 
           for model_name in models.keys()}

# Perform 20 iterations
n_iterations = 20

for i in range(n_iterations):
    for model_name, model in models.items():
        # Train the model
        model.fit(X_train, Y_train)
        
        # Get predictions
        y_pred = model.predict(X_test)
        
        # If the model supports probability predictions, compute ROC AUC
        if hasattr(model, "predict_proba"):
            y_pred_prob = model.predict_proba(X_test)[:, 1]
            roc_auc = roc_auc_score(Y_test, y_pred_prob)
        else:
            roc_auc = None  # Not all models support predict_proba
        
        # Calculate metrics
        accuracy = accuracy_score(Y_test, y_pred)
        f1 = f1_score(Y_test, y_pred)
        precision = precision_score(Y_test, y_pred)
        recall = recall_score(Y_test, y_pred)
        
        # Append metrics for this iteration
        results[model_name]["Accuracy"].append(accuracy)
        results[model_name]["F1 Score"].append(f1)
        results[model_name]["Precision"].append(precision)
        results[model_name]["Recall"].append(recall)
        if roc_auc is not None:
            results[model_name]["ROC AUC"].append(roc_auc)

# Calculate mean and standard deviation
final_results = {}
for model_name, metrics in results.items():
    final_results[model_name] = {
        metric: {
            "Mean": np.mean(values),
            "Std Dev": np.std(values)
        }
        for metric, values in metrics.items()
        if len(values) > 0  # Exclude ROC AUC if not available
    }

# Print results
for model_name, metrics in final_results.items():
    print(f"{model_name}:")
    for metric, stats in metrics.items():
        print(f"  {metric}: Mean = {stats['Mean']:.4f}, Std Dev = {stats['Std Dev']:.4f}")
    print()


/home/khuushi/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/khuushi/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/khuushi/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/khuushi/.local/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1105: DataConversionWarning: A column-vector y was passed when a 1d array was expe

SVM Linear:
  Accuracy: Mean = 0.8663, Std Dev = 0.0000
  F1 Score: Mean = 0.0398, Std Dev = 0.0000
  Precision: Mean = 0.5556, Std Dev = 0.0000
  Recall: Mean = 0.0207, Std Dev = 0.0000
  ROC AUC: Mean = 0.8329, Std Dev = 0.0000

SVM RBF:
  Accuracy: Mean = 0.8702, Std Dev = 0.0000
  F1 Score: Mean = 0.0714, Std Dev = 0.0000
  Precision: Mean = 0.9000, Std Dev = 0.0000
  Recall: Mean = 0.0372, Std Dev = 0.0000
  ROC AUC: Mean = 0.9462, Std Dev = 0.0000

SVM Sigmoid:
  Accuracy: Mean = 0.8125, Std Dev = 0.0000
  F1 Score: Mean = 0.0000, Std Dev = 0.0000
  Precision: Mean = 0.0000, Std Dev = 0.0000
  Recall: Mean = 0.0000, Std Dev = 0.0000
  ROC AUC: Mean = 0.5671, Std Dev = 0.0000

MLP:
  Accuracy: Mean = 0.9196, Std Dev = 0.0000
  F1 Score: Mean = 0.7627, Std Dev = 0.0000
  Precision: Mean = 0.6314, Std Dev = 0.0000
  Recall: Mean = 0.9628, Std Dev = 0.0000
  ROC AUC: Mean = 0.9759, Std Dev = 0.0000

XGBoost:
  Accuracy: Mean = 0.9506, Std Dev = 0.0000
  F1 Score: Mean = 0.8355, Std D